In [40]:

import numpy as np
import pandas as pd
import re
import csv
import threading
import multiprocessing
import os
import pickle

from _datetime import datetime
from sklearn.preprocessing import OneHotEncoder

In [41]:

product_headers_to_encode = ['PRODUCTTYPENAME', 'ROUTENAME', 'DOSAGEFORMNAME', 'MARKETINGCATEGORYNAME']
package_headers_to_encode = ['PACKAGEUNIT', 'PACKAGETYPE']

target_encoding = 'utf-8'
separ = '\t'

product_file = 'product.csv'
package_file = 'package.csv'

encoded_product_file = 'transformed_product_data.csv'
encoded_package_file = 'transformed_package_data.csv'

product_encode_file_exist = False
package_encode_file_exist = False


# TODO incohernce entre dates
# TODO incohernce entre routname / forme
# TODO incohernce entre valeurs numeric abberantes (ordre de grandeur)
# TODO incohernce entre valeurs phase et l'emballage
# TODO tester imputatin itérative
# TODO utiliser le one hot de sk learn au lieu de dummies de pandas

def assert_table_completeness(table):
    empty_cells = table.shape[0] - table.count(axis=0)
    unique_values = table.nunique(axis=0)

    print('Empty cells:\n{}\n'.format(empty_cells))
    print('Unique values:\n{}\n'.format(unique_values))


def assert_product_id_completeness(table, header):
    empty_cells = table.shape[0] - table.count(axis=0)
    unique_values = table.nunique(axis=0)

    try:
        assert empty_cells[header] == 0
        print('No empty values in the {} column'.format(header))
    except:
        print('There are {} empty values in the {} column'.format(empty_cells[header], header))
    try:
        assert unique_values[header] == table.shape[0]
        print('No duplicat values in the {} column'.format(header))
    except:
        print(
            'There are {} duplicat values in the {} column\n\n'.format(table.shape[0] - unique_values[header], header))


def get_unique_values(table, headers=''):
    uniques = {}
    if headers == '':
        cols = table.columns.values
        for n, c in enumerate(cols):
            uniques[c] = pd.unique(table[c])
    # else:
    #     uniques[headers] = pd.unique(table[headers])
    elif type(headers) is list:
        for header in headers:
            uniques[header] = pd.unique(table[header])
    elif type(headers) is str:
        uniques[headers] = pd.unique(table[headers])
    return uniques


def df_to_lower(table, columns='all'):
    cols = table.columns.values if columns == 'all' else columns
    for c in cols:
        try:
            table[c] = table[c].str.lower()
        except:
            pass


def get_decomposed_uniques(table, header):
    decomposed_uniques = {}
    if type(header) is str:
        for unique_header, uniques in get_unique_values(table, header).items():
            tmp_lst = []
            for val in uniques:
                if type(val) is str:
                    for decomposed in re.split(' ?[_|,;:<>/;] ?|^ | $', val):
                        if decomposed != '' and \
                                not decomposed in tmp_lst:
                            tmp_lst.append(decomposed)

            tmp_lst.sort()
            decomposed_uniques[unique_header] = tmp_lst
    else:
        raise TypeError('header should be a string representing a column header')

    return pd.DataFrame.from_dict(decomposed_uniques)


def get_onehot_encoders(table, cols):
    encoder_dict = {}
    for col in cols:
        uniques_vals = get_decomposed_uniques(table, header=col)
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit_transform(uniques_vals)
        encoder_dict[col] = enc
    return encoder_dict


def onehot_encode(table, header, threaded=False):
    # Create onehot codes for the specidfied column
    lst = []
    encoder_dict = get_onehot_encoders(table, [header])

    if threaded:
        threads = []

        class _onehot_encode_thread(threading.Thread):
            def __init__(self, threadID, name, start_index, end_index):
                threading.Thread.__init__(self)
                self.threadID = threadID
                self.name = name
                self.start_index = start_index
                self.end_index = end_index

            def run(self):
                _int_onehot_encode(self.start_index, self.end_index)

        def _int_onehot_encode(start_index, end_index):
            for index in np.int_(np.linspace(start_index, end_index, (end_index - start_index) + 1)):
                _tmp = np.zeros([1, len(encoder_dict[header].categories_[0])], dtype=int)
                if type(table.loc[index, header]) is str:
                    for decomposed in re.split('[_|,;:<>/;] ?|^ ', table.loc[index, header]):
                        _tmp |= np.int_(encoder_dict[header].transform([[decomposed]]).toarray())
                    lst.append(_tmp)

        cpu_count = multiprocessing.cpu_count()
        step = int(np.floor(table.shape[0] / cpu_count))
        rem = table.shape[0] % cpu_count;
        strt = 0

        for i in range(cpu_count):
            end = step * (i + 1)
            if i == cpu_count - 1:
                end += rem - 1
            thrd = _onehot_encode_thread(i, str(i), strt, end)
            thrd.start()
            threads.append(thrd)
            strt = end + 1

        count = 1
        for thrd in threads:
            thrd.join()
            # Update loading bar
            progress(1, 1, 'Thread {} finished'.format(count))
            print('')
            count += 1
    else:
        count = 0
        for index in range(table.shape[0]):
            _tmp = np.zeros([1, len(encoder_dict[header].categories_[0])], dtype=int)
            if type(table.loc[index, header]) is str:
                for decomposed in re.split('[_|,;:<>/;] ?|^ ', table.loc[index, header]):
                    _tmp |= np.int_(encoder_dict[header].transform([[decomposed]]).toarray())
                lst.append(_tmp)

            # Update loading bar
            if count == 1000:
                progress(index, table.shape[0])
                count = 0
            count += 1

    print(" -> Done", flush=True)

    # Replace dataframe column by encoded values
    table.loc[:, header] = pd.Series(lst)

    # return the encoder associated to that particular header
    return encoder_dict[header]


def time_methode(methode, status='', **kwargs):
    print('Timing {}'.format(methode.__name__))
    if status != '':
        print(status)
    start_time = datetime.now()
    print('Start time: {}'.format(start_time))
    ret = methode(**kwargs)
    end_time = datetime.now()
    print('End time: {}'.format(end_time))
    print('{} took: {}'.format(methode.__name__, (end_time - start_time)))
    if ret != '':
        return ret
    else:
        ret = 0
    return ret


def progress(count, total, status=''):
    bar_len = 50
    filled_len = int(round(bar_len * count / float(total)))
    _str = ''
    percents = np.ceil(100.0 * count / float(total))
    bar = '|' * filled_len + '_' * (bar_len - filled_len)

    if status == '':
        _str = '[{}] {}%'.format(bar, percents)
    else:
        _str = '[{}] {}% - {}'.format(bar, percents, status)

    print('\r', end='', flush=True)
    print(_str, end='', flush=True)


product_encode_file_exist = os.path.isfile(encoded_product_file)
package_encode_file_exist = os.path.isfile(encoded_package_file)

tmp_dic = {}

original_product_data = pd.read_csv(product_file, sep=';', encoding='latin1')
original_package_data = pd.read_csv(package_file, sep=';', encoding='latin1')

if product_encode_file_exist:
    print('Loading encoded product data from existing file...')
    product_data = pd.read_csv(encoded_product_file, sep=separ, encoding=target_encoding)

    # Populate onehot encoders dictionnary
    for header in product_headers_to_encode:
        tmp_dic[header] = pickle.load(open('{}_data_encoder.pkl'.format(header), 'rb'))
else:
    product_data = original_product_data

if package_encode_file_exist:
    print('Loading encoded package data from existing file...')
    package_data = pd.read_csv(encoded_package_file, sep=separ, encoding=target_encoding)

    # Populate onehot encoders dictionnary
    for header in package_headers_to_encode:
        tmp_dic[header] = pickle.load(open('{}_data_encoder.pkl'.format(header), 'rb'))
else:
    package_data = original_package_data

# Make everything lower characters in both tables
df_to_lower(product_data)
df_to_lower(package_data)

print('Get unique values for ROUTENAME column of PRODUCT table')
product_unique_values = get_decomposed_uniques(product_data, 'ROUTENAME')
print(product_unique_values)
print('Get unique values for each column of PACKAGING table')
package_unique_values = get_unique_values(package_data)

Loading encoded product data from existing file...
Loading encoded package data from existing file...
Get unique values for ROUTENAME column of PRODUCT table
                                             ROUTENAME
0    [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1    [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2    [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3    [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4    [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
..                                                 ...
167  [[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
168  [[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
169  [[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
170  [[0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
171  [[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...

[172 rows x 1 columns]
Get unique values for each column of PACKAGING table


# 1. Auscultation
## Etude des données du fichier 'package'

In [42]:
package_data.head()

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE,PACKAGESIZE,PACKAGEUNIT,PACKAGETYPE
0,0699-0155_60f08fe4-aacf-c815-e053-2991aa0a3ded,0699-0155,0699-0155-44,125 g in 1 canister,1900-01-01,NaN,n,n,125.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,0699-0311_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0311,0699-0311-16,"498 g in 1 bottle, plastic",1900-01-01,NaN,n,n,498.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,0699-0312_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0312,0699-0312-16,"498 g in 1 bottle, plastic",1900-01-01,NaN,n,n,498.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,0699-0317_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0317,0699-0317-16,"498 g in 1 bottle, plastic",1900-01-01,NaN,n,n,498.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,0699-0404_5bc1c974-e7b8-094c-e053-2991aa0ab726,0699-0404,0699-0404-01,28 g in 1 jar,1900-01-01,NaN,n,n,28.0,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [43]:
print('Assessing completnes of packaging data table')
assert_table_completeness(package_data)

Assessing completnes of packaging data table
Empty cells:
PRODUCTID                 41
PRODUCTNDC               193
NDCPACKAGECODE           193
PACKAGEDESCRIPTION         0
STARTMARKETINGDATE       683
ENDMARKETINGDATE      167431
NDC_EXCLUDE_FLAG           0
SAMPLE_PACKAGE             0
PACKAGESIZE                0
PACKAGEUNIT                0
PACKAGETYPE                0
dtype: int64

Unique values:
PRODUCTID              93084
PRODUCTNDC             91697
NDCPACKAGECODE        173692
PACKAGEDESCRIPTION     11501
STARTMARKETINGDATE      7370
ENDMARKETINGDATE         767
NDC_EXCLUDE_FLAG           1
SAMPLE_PACKAGE             2
PACKAGESIZE             2715
PACKAGEUNIT              110
PACKAGETYPE               55
dtype: int64



### Colonne PACKAGEDESCRIPTION
Cette colonne est présentée sous forme de phrase et contient de multiples informations. Le volume, son unité, le nombre 
de contenant et son type. S'il existe plusieurs contenants pour un objet, ils sont concaténés par un séparateur '>' de 
manière hiérarchique.

### Colonne STARTMARKETINGDATE
Les valeurs de cette colonne sont de type date. Elles ont l'air séquentielles. 

### Colonne ENDMARKETINGDATE
Les valeurs de cette colonne sont de type date. Il existe beaucoup de valeurs manquantes.

### Colonne NDC_EXCLUDE_FLAG

In [44]:
package_data['NDC_EXCLUDE_FLAG'].unique()

array(['n'], dtype=object)

Tous les objets possèdent la valeur N et ne présente aucune valeur manquante. 

### Colonne SAMPLE_PACKAGE

In [45]:
package_data['SAMPLE_PACKAGE'].value_counts()

n    173223
y       664
Name: SAMPLE_PACKAGE, dtype: int64

Les valeurs possibles sont 'Y' ou 'N'. Il y a une majorité de 'N' et aucune valeur manquante.

## Etude des données du fichier 'product'

In [46]:
product_data.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,0699-0155_60f08fe4-aacf-c815-e053-2991aa0a3ded,0699-0155,[[0 0 1 0 0 0 0]],topex neutral fluoride foam,NaN,sodium fluoride,[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1900-01-01,NaN,[[0 0 0 0 0 0 0 1 0 0]],NaN,dentsply llc. professional division trading as...,sodium fluoride,.02,g/g,NaN,NaN,n,2020-12-31
1,0699-0311_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0311,[[0 0 1 0 0 0 0]],topex 60 second fluoride gel,NaN,sodium fluoride,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1900-01-01,NaN,[[0 0 0 0 0 0 0 1 0 0]],NaN,dentsply llc. professional division trading as...,sodium fluoride,25.9,mg/g,NaN,NaN,n,2020-12-31
2,0699-0312_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0312,[[0 0 1 0 0 0 0]],topex 60 second fluoride gel,NaN,sodium fluoride,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1900-01-01,NaN,[[0 0 0 0 0 0 0 1 0 0]],NaN,dentsply llc. professional division trading as...,sodium fluoride,25.9,mg/g,NaN,NaN,n,2020-12-31
3,0699-0317_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0317,[[0 0 1 0 0 0 0]],topex 60 second fluoride gel,NaN,sodium fluoride,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1900-01-01,NaN,[[0 0 0 0 0 0 0 1 0 0]],NaN,dentsply llc. professional division trading as...,sodium fluoride,25.9,mg/g,NaN,NaN,n,2020-12-31
4,0699-0404_5bc1c974-e7b8-094c-e053-2991aa0ab726,0699-0404,[[0 0 1 0 0 0 0]],dry socket,NaN,guaiacol and eugenol,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1900-01-01,NaN,[[0 0 0 0 0 0 0 1 0 0]],NaN,"dentsply llc, professional division, trading a...",guaiacol; eugenol,.0416; .0416,g/g; g/g,"standardized chemical allergen [epc],increased...",NaN,n,2020-12-31


In [47]:
print('Assessing completnes product data table')
assert_table_completeness(product_data)

Assessing completnes product data table
Empty cells:
PRODUCTID                               0
PRODUCTNDC                              0
PRODUCTTYPENAME                         0
PROPRIETARYNAME                         6
PROPRIETARYNAMESUFFIX               83075
NONPROPRIETARYNAME                      4
DOSAGEFORMNAME                          0
ROUTENAME                            1932
STARTMARKETINGDATE                      0
ENDMARKETINGDATE                    88916
MARKETINGCATEGORYNAME                   0
APPLICATIONNUMBER                   13097
LABELERNAME                           557
SUBSTANCENAME                        2309
ACTIVE_NUMERATOR_STRENGTH            2309
ACTIVE_INGRED_UNIT                   2309
PHARM_CLASSES                       51009
DEASCHEDULE                         88815
NDC_EXCLUDE_FLAG                        0
LISTING_RECORD_CERTIFIED_THROUGH     4325
dtype: int64

Unique values:
PRODUCTID                           93238
PRODUCTNDC                          

### Colonne PRODUCTTYPENAME

In [48]:
product_data['PRODUCTTYPENAME'].value_counts()

[[0 1 0 0 0 0 0]]    46172
[[0 0 1 0 0 0 0]]    44526
[[0 0 0 1 0 0 0]]     2008
[[0 0 0 0 1 0 0]]      294
[[0 0 0 0 0 1 0]]      124
[[0 0 0 0 0 0 1]]      108
[[1 0 0 0 0 0 0]]        6
Name: PRODUCTTYPENAME, dtype: int64

Il y a 7 valeurs possibles textuelles catégorielles dans cette colonne.

### Colonne PROPRIETARYNAME

In [49]:
product_data['PROPRIETARYNAME'].nunique()

29775

In [50]:
# product_data['PROPRIETARYNAME'][393:401]

Dans cette colonne, il existe un grand nombre de valeurs différentes. Les mêmes valeurs peuvent être présentes sous 
différentes formes notamment en minuscules ou majuscules, il existe donc une inconsistance entre les valeurs.

### Colonne PROPRIETARYNAMESUFFIX

In [51]:
product_data['PROPRIETARYNAMESUFFIX'].nunique()

3726

Dans cette colonne, il y a un nombre important de valeurs manquantes. Ces valeurs sont textuelles et il existe un nombre 
élevé de valeurs différentes.

### Colonne NONPROPRIETARYNAME

In [52]:
product_data['NONPROPRIETARYNAME'].nunique()

13104

In [53]:
product_data['NONPROPRIETARYNAME'][2:6]

2         sodium fluoride
3         sodium fluoride
4    guaiacol and eugenol
5         sodium fluoride
Name: NONPROPRIETARYNAME, dtype: object

Cette colonne présente seulement 4 valeurs manquantes. Ceux sont des données textuelles inconsistantes, par exemple 
pouvant représenter la même valeur en caratères minuscules ou majuscules. Il y a un nombre très important de valeurs 
différentes.

### Colonne DOSAGEFORMNAME

In [54]:
product_data['DOSAGEFORMNAME'].value_counts()

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]]    15442
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]]     9387
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]     8764
[[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]     5104
[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

Cette colonne contient 134 différentes valeurs textuelles. Comme on peut le voir, différentes catégories peuvent être 
affectées au même objet. La colonne ne présente aucune valeur manquante.

### Colonne ROUTENAME

In [55]:
product_data['ROUTENAME'].value_counts()

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]    54716
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]]    21558
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]     3374
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]     1315
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]     1257
                                                                                                                                            ...  
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0

Cette colonne contient 180 différentes valeurs textuelles. Comme on peut le voir, différentes catégories peuvent être 
affectées au même objet. La colonne présente 1932 valeurs manquantes.

### Colonne STARTMARKETINGDATE
Les valeurs sont de type date, il n'y a aucune valeur manquante. 

### Colonne ENDMARKETINGDATE
Les valeurs sont de type date, il y a un grand nombre de valeurs manquantes. 

### Colonne MARKETINGCATEGORYNAME

In [56]:
product_data['MARKETINGCATEGORYNAME'].value_counts()
# %%
"""
Les valeurs sont de type textuelles, il y a 26 catégories différentes et ne présente aucune valeur manquante.  
"""
# %%
"""
### Colonne APPLICATIONNUMBER
"""
# %%
product_data['APPLICATIONNUMBER'].nunique()
# %%
"""
Cette colonne spécifie le numéro de série de la catégorie marketing. Le nombre de valeurs manquantes est élevé, et 
comme il y a un numéro de série pour chaque objet dans une catégorie, le nombre de valeurs différentes est également 
important.
"""

'\nCette colonne spécifie le numéro de série de la catégorie marketing. Le nombre de valeurs manquantes est élevé, et \ncomme il y a un numéro de série pour chaque objet dans une catégorie, le nombre de valeurs différentes est également \nimportant.\n'

### Colonne LABELERNAME

In [57]:
product_data['LABELERNAME'].nunique()

6095

In [58]:
product_data['LABELERNAME'][7291:7293]
# %%
"""
La colonne présente peu de valeurs manquantes (557). Si on remarque qu'il existe un nombre important de valeurs 
différentes, les données sont cependant inconsistantes.
"""

"\nLa colonne présente peu de valeurs manquantes (557). Si on remarque qu'il existe un nombre important de valeurs \ndifférentes, les données sont cependant inconsistantes.\n"

### Colonne SUBSTANCENAME

In [59]:
product_data['SUBSTANCENAME'].nunique()

8976

In [60]:
product_data['SUBSTANCENAME'][727:735]

727    xanthium strumarium pollen; plantago lanceolat...
728    xanthium strumarium pollen; plantago lanceolat...
729    alnus rhombifolia pollen; fraxinus americana p...
730    alnus rhombifolia pollen; fraxinus americana p...
731    alnus rhombifolia pollen; fraxinus americana p...
732       betula nigra pollen; betula populifolia pollen
733    gibberella zeae; aspergillus niger var. niger;...
734       carya ovata pollen; carya illinoinensis pollen
Name: SUBSTANCENAME, dtype: object

Cette colonne présente un nombre assez important de données manquantes (2309), ceux sont des données textuelles 
catégorielles. Or le nombre de catégories parait élevé, comme le montre le nombre de valeurs uniques. Chaque objet 
peut cependant présenté plusieurs catégories séparées par un ';'.

### Colonne ACTIVE_NUMERATOR_STRENGTH

In [61]:
product_data['ACTIVE_NUMERATOR_STRENGTH'][725:731]

725                                                  .05
726                                                   .1
727                                   .05; .05; .05; .05
728                           40000; 40000; 40000; 40000
729          .05; .05; .05; .05; .05; .05; .05; .05; .05
730    40000; 40000; 40000; 40000; 40000; 40000; 4000...
Name: ACTIVE_NUMERATOR_STRENGTH, dtype: object

Ceux sont des données numériques qui paraissent dupliquées pour le même objet.  

### Colonne ACTIVE_INGRED_UNIT

In [62]:
product_data['ACTIVE_INGRED_UNIT'][725:731]

725                                                 g/ml
726                                                 g/ml
727                               g/ml; g/ml; g/ml; g/ml
728               [pnu]/ml; [pnu]/ml; [pnu]/ml; [pnu]/ml
729    g/ml; g/ml; g/ml; g/ml; g/ml; g/ml; g/ml; g/ml...
730    [pnu]/ml; [pnu]/ml; [pnu]/ml; [pnu]/ml; [pnu]/...
Name: ACTIVE_INGRED_UNIT, dtype: object

Ceux sont des données textuelles catégorielles qui présentent l'unité de la colonne 'ACTIVE_INGRED_UNIT'. Les données
paraissent également dupliquées pour le même objet.

### Colonne PHARM_CLASSES

In [63]:
product_data['PHARM_CLASSES'].nunique()

1285

In [64]:
product_data['PHARM_CLASSES'][725]

'non-standardized pollen allergenic extract [epc],increased histamine release [pe],cell-mediated immunity [pe],increased igg production [pe],pollen [cs],allergens [cs]'

Ceux sont des données textuelles catégorielles présentant plusieurs catégories pour un même objet. Il y a un grand 
nombre de valeurs manquantes.
"""
# %%
"""
### Colonne DEASCHEDULE

In [65]:
product_data['DEASCHEDULE'].value_counts()

cii     1802
civ     1749
ciii     479
cv       393
Name: DEASCHEDULE, dtype: int64

Cette colonne présente un nombre important de données manquantes. Ceux sont des données catégorielles, présentant 
seulement 4 catégories.

### Colonne NDC_EXCLUDE_FLAG

In [66]:
product_data['NDC_EXCLUDE_FLAG'].value_counts()

n    93238
Name: NDC_EXCLUDE_FLAG, dtype: int64

Cette colonne présente seulement une catégorie 'N'.

# 2. Relations entre attributs
## Informations communes
Les colonnes 'PRODUCTID' des tables 'package' et 'product' contiennent deux informations concaténées: l'id du produit 
ainsi que le contenu de leur colonne 'PRODUCTNDC', le code label et le code segment produit.  
Dans la documentation NDC, il est précisé que c'est pour prévenir le duplicata de lignes.

La colonne 'NDCPACKAGECODE' de la table 'package' contient deux informations concaténées: le code segment du package et 
le contenu de la colonne 'PRODUCTNDC', le code label et le code segment produit.

La colonne 'PACKAGEDESCRIPTION' de la table 'package' contient plusieurs informations concaténées. En plus des 
informations propres à la description du package, il y a dans la majorité des objets la valeur 'NDCPACKAGECODE' associée
.

La colonne 'APPLICATIONNUMBER' de la table 'product' présente la majorité du temps le contenu de la colonne 
'MARKETINGCATEGORYNAME' et spécifie son numéro de série.

Dans les deux tables, il existe des colonnes 'STARTMARKETINGDATE',  'ENDMARKETINGDATE' et 'NDCEXLUDEDFLAG'. 
Elles semblent présenter les mêmes informations.

## Corrélation
Pour la table 'product':
Il semble pouvoir exister une corrélation entre les attributs 'ROUTENAME' et 'DOSAGEFORMNAME' qui présentent des idées 
d'administration similaires. 
On peut également considérer l'existance d'une corrélation entre les modes d'administration
et les dosages du médicament, donc les attributs 'ROUTENAME', 'DOSAGEFORMNAME' et ceux 'ACTIVE_NUMERATOR_STRENGTH', 
'ACTIVE_INGRED_UNIT'.
L'attribut 'PHARM_CLASS' semble pouvoir être corrélé à l'attribut 'SUBSTANCENAME'.

# 3. Correction des incohérences
On élimine dans un premier temps les duplicata de valeurs dans les attributs 'ACTIVE_NUMERATOR_STRENGTH', 
'ACTIVE_INGRED_UNIT' de la table 'product'. 
## Table 'product'

In [67]:

# TODO: keep most frequent value
# dupl_val_cols = ['ACTIVE_NUMERATOR_STRENGTH', 'ACTIVE_INGRED_UNIT']
# for c in dupl_val_cols:
#     product_data[c] = product_data[c].replace(to_replace=r'\;.*', value='', regex=True)


Il existerait également une incohérence si l'attribut 'ENDMARKETINGDATE' est moins récent que le 'STARTMARKETINGDATE'.
On vérifie s'il en existe dans les tables 'product' et 'package'.

In [68]:

# conversion to datetime format
def date_convert():
    date_cols = ['STARTMARKETINGDATE', 'ENDMARKETINGDATE', 'LISTING_RECORD_CERTIFIED_THROUGH']
    for c in date_cols:
        product_data[c] = pd.to_datetime(product_data[c], errors='coerce', format='%Y%m%d')


if not product_encode_file_exist:
    time_methode(date_convert)

# compare STARTMARKETINGDATE and ENDMARKETINGDATE
# replace ENDMARKETINGDATE to NaT when incoherence
product_data.loc[
    (product_data['STARTMARKETINGDATE'] > product_data['ENDMARKETINGDATE']), 'ENDMARKETINGDATE'] = pd.NaT

## Table 'package'
La colonne 'PACKAGEDESCRIPTION' contient beaucoup trop d'informations pour être exploitable. Tout d'abord, on garde 
seulement l'information du package le plus informatif (le dernier) car spécifie le volume le plus précis.
On supprime l'information dupliquée du 'NDCPACKAGECODE'. 
Enfin, on crée une colonne pour chaque information: 'PACKAGESIZE', 'PACKAGEUNIT' et 'PACKAGETYPE'.
On peut retirer la colonne 'PACKAGEDESCRIPTION' de la table.

In [69]:

if not package_encode_file_exist:
    # keep only most informative packaging and remove duplicate info NDCPACKAGECODE
    package_data['PACKAGEDESCRIPTION'] = package_data['PACKAGEDESCRIPTION'].replace(to_replace=r'.*(\>|\*\ ) |\(.*',
                                                                                    value='', regex=True)

    # split info into multiple columns
    search = {0: [], 1: [], 2: []}
    for values in package_data['PACKAGEDESCRIPTION']:
        s = re.search(r'(^\.?[0-9\.]+)\ (.*)\ in\ 1\ (.*)', values)
        for i in range(3):
            search[i].append(s.group(i + 1))

    for i, n in enumerate(['PACKAGESIZE', 'PACKAGEUNIT', 'PACKAGETYPE']):
        package_data[n] = search[i]

Traitement des colonnes 'STARTMARKETINGDATE', 'ENDMARKETINGDATE' similairement à la table 'product'.

In [70]:

if not package_encode_file_exist:
    # conversion to datetime format
    date_cols = ['STARTMARKETINGDATE', 'ENDMARKETINGDATE']
    for c in date_cols:
        package_data[c] = pd.to_datetime(package_data[c], errors='coerce', format='%Y%m%d')

    # compare STARTMARKETINGDATE and ENDMARKETINGDATE
    # replace ENDMARKETINGDATE to NaT when incoherence
    package_data.loc[
        (package_data['STARTMARKETINGDATE'] > package_data['ENDMARKETINGDATE']), 'ENDMARKETINGDATE'] = pd.NaT

# 4. Données manquantes
## Table 'package'
On s'intéresse aux données manquantes dans les colonnes PRODUCTID, PRODUCTNDC, NDCPACKAGECODE.

In [71]:
if not package_encode_file_exist:
    package_missing_ndcpackagecode = package_data.iloc[np.where(pd.isnull(package_data['NDCPACKAGECODE']))]
    values = package_missing_ndcpackagecode['PACKAGEDESCRIPTION'].str.extract(r'\((.*?)\).*')
    for index, row in values.iterrows():
        package_data.loc[index, 'NDCPACKAGECODE'] = row[0]

In [72]:
if not package_encode_file_exist:
    package_missing_productndc = package_data.iloc[np.where(pd.isnull(package_data['PRODUCTNDC']))]
    values = package_missing_productndc['NDCPACKAGECODE'].str.extract(r'^([\w]+-[\w]+)')
    for index, row in values.iterrows():
        package_data.loc[index, 'PRODUCTNDC'] = row[0]

In [73]:

if not package_encode_file_exist:
    # TODO : find a way to retrieve PRODUCTID from 'product' table
    package_missing_ndcproductid = package_data.iloc[np.where(pd.isnull(package_data['PRODUCTID']))]

Il existe des valeurs manquantes pour les colonnes 'STARTMARKETINGDATE' et 'ENDMARKETINGDATE' dans la table 'package'
mais on choisit de ne pas les compléter car on ne peut effectuer d'estimation précise. 

## Table 'product'

# 5. Duplications données

In [74]:

# TODO: drop column PACKAGEDESCRIPTION

# Transformation en données numériques (après question 8)
## Table 'package'

## Table 'product'

In [75]:

# %%
# TODO: hash PROPRIETARYNAME NONPROPRIETARYNAME LABELERNAME PROPRIETARYNAMESUFFIX
# TODO: separate and hash SUBSTANCENAME PHARM_CLASSES
# TODO : split ACTIVE_INGRED_UNIT by '/' (nan others), then one hot each col

# TODO: ideas?? APPLICATIONNUMBER
# %%
# TODO : analysis ratio per category

## Encodage onehot

In [76]:

# Call and time onehot encoding for a column
if not product_encode_file_exist:
    for header in product_headers_to_encode:
        kwargs = dict(table=product_data, header=header)
        tmp_dic[header] = time_methode(onehot_encode, header, **kwargs)
        pickle.dump(tmp_dic[header], open('{}_data_encoder.pkl'.format(header), 'wb'), pickle.HIGHEST_PROTOCOL)

if not package_encode_file_exist:
    for header in package_headers_to_encode:
        kwargs = dict(table=package_data, header=header)
        tmp_dic[header] = time_methode(onehot_encode, header, **kwargs)
        pickle.dump(tmp_dic[header], open('{}_data_encoder.pkl'.format(header), 'wb'), pickle.HIGHEST_PROTOCOL)

for header, enc in tmp_dic.items():
    file = open('Encoding_{}.txt'.format(header), 'w')
    for category in enc.categories_[0]:
        tmp_str = str(enc.transform([[category]]).toarray())
        tmp_str = category + ' ' * (40 - len(category)) + tmp_str.replace('\n', '\n' + ' ' * 40) + '\n'
        file.write(tmp_str)
    file.close()

# Save transformed data to file
if not product_encode_file_exist:
    kwargs = dict(path_or_buf= encoded_product_file, index= False, sep= separ, encoding= target_encoding,
              quoting= csv.QUOTE_NONNUMERIC)
    time_methode(product_data.to_csv, **kwargs)

if not product_encode_file_exist:
    kwargs = dict(path_or_buf=encoded_package_file, index=False, sep=separ, encoding=target_encoding,
                  quoting=csv.QUOTE_NONNUMERIC)
    time_methode(package_data.to_csv, **kwargs)

## Résultats

In [77]:
print('Encoded product data:')
product_data

Encoded product data:


,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,0699-0155_60f08fe4-aacf-c815-e053-2991aa0a3ded,0699-0155,[[0 0 1 0 0 0 0]],topex neutral fluoride foam,NaN,sodium fluoride,[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1900-01-01,NaN,[[0 0 0 0 0 0 0 1 0 0]],NaN,dentsply llc. professional division trading as...,sodium fluoride,.02,g/g,NaN,NaN,n,2020-12-31
1,0699-0311_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0311,[[0 0 1 0 0 0 0]],topex 60 second fluoride gel,NaN,sodium fluoride,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1900-01-01,NaN,[[0 0 0 0 0 0 0 1 0 0]],NaN,dentsply llc. professional division trading as...,sodium fluoride,25.9,mg/g,NaN,NaN,n,2020-12-31
2,0699-0312_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0312,[[0 0 1 0 0 0 0]],topex 60 second fluoride gel,NaN,sodium fluoride,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1900-01-01,NaN,[[0 0 0 0 0 0 0 1 0 0]],NaN,dentsply llc. professional division trading as...,sodium fluoride,25.9,mg/g,NaN,NaN,n,2020-12-31
3,0699-0317_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0317,[[0 0 1 0 0 0 0]],topex 60 second fluoride gel,NaN,sodium fluoride,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1900-01-01,NaN,[[0 0 0 0 0 0 0 1 0 0]],NaN,dentsply llc. professional division trading as...,sodium fluoride,25.9,mg/g,NaN,NaN,n,2020-12-31
4,0699-0404_5bc1c974-e7b8-094c-e053-2991aa0ab726,0699-0404,[[0 0 1 0 0 0 0]],dry socket,NaN,guaiacol and eugenol,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1900-01-01,NaN,[[0 0 0 0 0 0 0 1 0 0]],NaN,"dentsply llc, professional division, trading a...",guaiacol; eugenol,.0416; .0416,g/g; g/g,"standardized chemical allergen [epc],increased...",NaN,n,2020-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93233,70710-1302_149be946-a75a-4ebb-94ff-e2cc8e5b0bfc,70710-1302,[[0 0 1 0 0 0 0]],mesalamine,NaN,mesalamine,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,NaN,2020-02-14,NaN,[[1 0 0 0 0 0 0 0 0 0]],anda208953,zydus pharmaceuticals (usa) inc.,mesalamine,1000,mg/1,"aminosalicylate [epc],aminosalicylic acids [cs]",NaN,n,2021-12-31
93234,70771-1513_fea66d66-0735-4917-ba39-d9e02a894717,70771-1513,[[0 0 1 0 0 0 0]],mesalamine,NaN,mesalamine,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,NaN,2020-02-14,NaN,[[1 0 0 0 0 0 0 0 0 0]],anda208953,cadila healthcare limited,mesalamine,1000,mg/1,"aminosalicylate [epc],aminosalicylic acids [cs]",NaN,n,2021-12-31
93235,73598-0651_b19ce6d2-6875-4bd6-b8b9-b844db5503e9,73598-0651,[[0 1 0 0 0 0 0]],isopropyl alcohol,NaN,isopropyl alcohol,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,NaN,2020-02-14,NaN,[[0 0 0 0 0 1 0 0 0 0]],part333a,jhk inc,isopropyl alcohol,700,ml/l,NaN,NaN,n,2021-12-31
93236,73598-0661_9b83d918-bc9d-4ef6-815b-71ba9340f86a,73598-0661,[[0 1 0 0 0 0 0]],hydrogen peroxide,NaN,hydrogen peroxide,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,NaN,2020-02-14,NaN,[[0 0 0 0 0 1 0 0 0 0]],part333a,jhk inc,hydrogen peroxide,30,mg/ml,NaN,NaN,n,2021-12-31


In [78]:
print('Encoded packaging data:')
package_data

Encoded packaging data:


,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE,PACKAGESIZE,PACKAGEUNIT,PACKAGETYPE
0,0699-0155_60f08fe4-aacf-c815-e053-2991aa0a3ded,0699-0155,0699-0155-44,125 g in 1 canister,1900-01-01,NaN,n,n,125.0000,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,0699-0311_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0311,0699-0311-16,"498 g in 1 bottle, plastic",1900-01-01,NaN,n,n,498.0000,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,0699-0312_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0312,0699-0312-16,"498 g in 1 bottle, plastic",1900-01-01,NaN,n,n,498.0000,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,0699-0317_59039cf6-9008-7367-e053-2a91aa0a2fe9,0699-0317,0699-0317-16,"498 g in 1 bottle, plastic",1900-01-01,NaN,n,n,498.0000,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,0699-0404_5bc1c974-e7b8-094c-e053-2991aa0ab726,0699-0404,0699-0404-01,28 g in 1 jar,1900-01-01,NaN,n,n,28.0000,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...,...,...,...,...,...,...,...,...,...
173882,70710-1302_149be946-a75a-4ebb-94ff-e2cc8e5b0bfc,70710-1302,70710-1302-7,6 suppository in 1 blister pack,2020-02-14,NaN,n,n,6.0000,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
173883,70771-1513_fea66d66-0735-4917-ba39-d9e02a894717,70771-1513,70771-1513-7,6 suppository in 1 blister pack,2020-02-14,NaN,n,n,6.0000,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
173884,73598-0651_b19ce6d2-6875-4bd6-b8b9-b844db5503e9,73598-0651,73598-0651-1,".0591 l in 1 bottle, spray",2020-02-14,NaN,n,n,0.0591,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
173885,73598-0661_9b83d918-bc9d-4ef6-815b-71ba9340f86a,37598-0661,73598-0661-1,"59.1 ml in 1 bottle, spray",2020-02-14,NaN,n,n,59.1000,[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
